In [1]:
#### set directories ####
results_filename <- "../results/audit_stats.csv"

#### load packages ####
packages = c('reticulate', 'tidyverse', 'stringr', 'kableExtra')
for (pkg in packages) {
    library(pkg, character.only = TRUE, warn.conflicts = FALSE, quietly = TRUE, verbose = FALSE)
}
options(dplyr.width = Inf, dplyr.print_max = 1e9)
options(stringsAsFactors = FALSE)

Warning message in system("timedatectl", intern = TRUE):
“running command 'timedatectl' had status 1”
── Attaching core tidyverse packages ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.0     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [2]:
#### script parameters ####
all_data_names = c("fico", "givemecredit", "german")
all_actionset_names = c("simple_1D", "complex_1D", "complex_nD")
all_method_names = c("reach", "ar", "dice")
all_model_types = c("logreg", "xgb", "rf")

#### embellishments ####

EMPTY_TEX_STRING = "---"

ACTIONSET_TITLES = c(
    'complex_nD' = 'Actual',
    'complex_1D' = 'Separable',
    'simple_1D' = 'Simple'
)

METHOD_TITLES = c(
    'reach' = '\\reach{}',
    'ar' = '\\ar{}',
    'dice' = '\\dice{}'
)

MODEL_TYPE_TITLES = c(
    'logreg' = '\\LR{}',
    'rf' = '\\RF{}',
    'xgb' = '\\XGB{}'
)

DATASET_TITLES = c(
    'fico' = '\\ficoinfo{}',
    'givemecredit' = '\\givemecredit{}',
    'german' = '\\germaninfo{}'
)

METRIC_TITLES = c(
    'certifies_infeasibility_cnt' = 'Certifies no recourse',
    'abstains_cnt' = 'Abstains/inconclusive',
    'finds_action_cnt' = 'Finds action',
    'finds_good_action_cnt' = 'Finds valid action',
    'loophole_cnt' = 'Loophole',
    'finds_no_action_cnt' = 'Finds no action',
    'blindspot_cnt' = 'Blindspot'
)

# select metrics for the cells here
# change order of entries to change order in cell
cell_metric_names = c(
    'certifies_infeasibility_cnt',
    'abstains_cnt',
    'finds_action_cnt',
    'loophole_cnt',
    'finds_no_action_cnt',
    'blindspot_cnt'
)

In [3]:
# #### load data ####
# filter raw results to the datasets and actionsets that you care about
raw_df = read.csv(results_filename) %>%
    filter(data_name %in% all_data_names,
           actionset_name %in% all_actionset_names)
    # basic changes in stat names
    # mutate(stat_name = str_replace(stat_name, "recourse_true_", "recourse_"))

# raw_df = bind_rows(
#     raw_df %>% filter(!grepl("recourse_", stat_name)),
#     raw_df %>% filter(grepl("recourse_", stat_name)) %>% mutate(stat_name = str_replace(stat_name, "_cnt", paste0("_", label_type, "_cnt")))
# )

tmp_df = raw_df %>%
    filter(stat_name == "n", label_type %in% c("pos","neg"), prediction_type %in% c("pos","neg")) %>%
    mutate(n = stat_value) %>%
    select(-starts_with("stat_"), -method_name) %>%
    group_by(data_name, actionset_name, model_type, label_type, prediction_type) %>%
    slice(1) %>%
    ungroup()

samples_df = bind_rows(
    tmp_df,
    # tmp_df %>% group_by(data_name, actionset_name, model_type, label_type) %>% tally(wt = n) %>% mutate(prediction_type = "all"),
    # tmp_df %>% group_by(data_name, actionset_name, model_type, prediction_type) %>% tally(wt = n) %>% mutate(label_type = "all"),
)

samples_df

label_type,prediction_type,data_name,actionset_name,model_type,n
<chr>,<chr>,<chr>,<chr>,<chr>,<int>
neg,neg,fico,complex_1D,logreg,2347
neg,pos,fico,complex_1D,logreg,769
pos,neg,fico,complex_1D,logreg,933
pos,pos,fico,complex_1D,logreg,1793
neg,neg,fico,complex_1D,rf,2553
neg,pos,fico,complex_1D,rf,563
pos,neg,fico,complex_1D,rf,855
pos,pos,fico,complex_1D,rf,1871
neg,neg,fico,complex_1D,xgb,2355


In [4]:
#### build table of recourse stats ####
recourse_cnt_df = raw_df %>%
    filter(actionset_name %in% all_actionset_names,
           model_type %in% all_model_types,
           # grepl("recourse_", stat_name))
           stat_name %in% cell_metric_names
    )

# add normalized metrics
recourse_pct_df = recourse_cnt_df %>%
    left_join(samples_df) %>%
    mutate(stat_value = stat_value / n,
           stat_name = str_replace(stat_name, "_cnt", "_pct")) %>%
    select(-n)

recourse_df = bind_rows(recourse_cnt_df, recourse_pct_df) %>%
    select(data_name, actionset_name, model_type, method_name, everything())

recourse_df

Joining with `by = join_by(label_type, prediction_type, data_name, actionset_name, model_type)`


data_name,actionset_name,model_type,method_name,stat_name,stat_value,label_type,prediction_type
<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>
fico,complex_nD,rf,reach,finds_action_cnt,1871,pos,pos
fico,complex_nD,rf,reach,finds_no_action_cnt,0,pos,pos
fico,complex_nD,rf,reach,certifies_infeasibility_cnt,0,pos,pos
fico,complex_nD,rf,reach,abstains_cnt,0,pos,pos
fico,complex_nD,rf,reach,loophole_cnt,0,pos,pos
fico,complex_nD,rf,reach,blindspot_cnt,0,pos,pos
fico,complex_nD,rf,reach,finds_action_cnt,658,pos,neg
fico,complex_nD,rf,reach,finds_no_action_cnt,197,pos,neg
fico,complex_nD,rf,reach,certifies_infeasibility_cnt,197,pos,neg


In [5]:
#### create table stats
table_stats_df = recourse_df %>%
    filter(prediction_type == "neg", label_type == "neg", endsWith(stat_name, "pct")) %>%
    mutate(svalue = sprintf("%1.0f", stat_value),
           svalue_pct = sprintf("%1.1f\\%%", 100 * stat_value),
           svalue_dec = sprintf("%1.3f", stat_value)) %>%
    mutate(svalue = ifelse(str_detect(stat_name, "_pct"), svalue_pct, svalue),
           svalue = ifelse(is.na(stat_value), EMPTY_TEX_STRING, svalue)) %>%
    mutate(svalue = ifelse(method_name != "reach" & stat_name == "certifies_infeasibility_pct",
                           EMPTY_TEX_STRING, svalue)) %>%
    select(-svalue_pct, -svalue_dec)

table_stats_df

data_name,actionset_name,model_type,method_name,stat_name,stat_value,label_type,prediction_type,svalue
<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>
fico,complex_nD,rf,reach,finds_action_pct,0.6588327458,neg,neg,65.9\%
fico,complex_nD,rf,reach,finds_no_action_pct,0.3411672542,neg,neg,34.1\%
fico,complex_nD,rf,reach,certifies_infeasibility_pct,0.3411672542,neg,neg,34.1\%
fico,complex_nD,rf,reach,abstains_pct,0.0000000000,neg,neg,0.0\%
fico,complex_nD,rf,reach,loophole_pct,0.0000000000,neg,neg,0.0\%
fico,complex_nD,rf,reach,blindspot_pct,0.0000000000,neg,neg,0.0\%
givemecredit,complex_nD,logreg,dice,finds_action_pct,1.0000000000,neg,neg,100.0\%
givemecredit,complex_nD,logreg,dice,finds_no_action_pct,0.0000000000,neg,neg,0.0\%
givemecredit,complex_nD,logreg,dice,certifies_infeasibility_pct,0.0000000000,neg,neg,---


In [6]:
cells_df = table_stats_df %>%
    # filter(model_type %in% c("finds_action_pct", "finds_no_action_pct", "certifies_pct", "loophole_pct", "blindspot_pct")) %>%
    arrange(data_name, actionset_name, model_type, method_name) %>%
    select(-stat_value, -label_type, -prediction_type) %>%
    pivot_wider(
        names_from = stat_name,
        values_from = svalue
    )

head(cells_df)

data_name,actionset_name,model_type,method_name,finds_action_pct,finds_no_action_pct,certifies_infeasibility_pct,abstains_pct,loophole_pct,blindspot_pct
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
fico,complex_1D,logreg,ar,83.3\%,16.7\%,---,0.0\%,41.5\%,0.0\%
fico,complex_1D,logreg,dice,51.6\%,48.4\%,---,0.0\%,32.3\%,23.5\%
fico,complex_1D,logreg,reach,74.2\%,25.8\%,25.8\%,0.0\%,0.0\%,0.0\%
fico,complex_1D,rf,dice,45.9\%,54.1\%,---,0.0\%,26.8\%,20.3\%
fico,complex_1D,rf,reach,65.9\%,34.1\%,34.1\%,0.0\%,0.0\%,0.0\%
fico,complex_1D,xgb,dice,52.4\%,47.6\%,---,0.0\%,38.6\%,22.7\%


In [7]:
table_df = cells_df %>%
    filter(actionset_name == "complex_nD") %>%
    # Add bolds and colors
    mutate(loophole_pct = ifelse(
        method_name == "reach",
        sprintf("\\textbf{%s}", loophole_pct),
        loophole_pct)
    ) %>%
    mutate(loophole_pct = ifelse(
        method_name != "reach" & loophole_pct != "0.0\\%",
        sprintf("\\textcolor{\\pitfall}{%s}", loophole_pct),
        loophole_pct)
    ) %>%
    mutate(blindspot_pct = ifelse(
        method_name == "reach",
        sprintf("\\textbf{%s}", blindspot_pct),
        blindspot_pct)
    ) %>%
    mutate(blindspot_pct = ifelse(
        method_name != "reach" & blindspot_pct != "0.0\\%",
        sprintf("\\textcolor{\\pitfall}{%s}", blindspot_pct),
        blindspot_pct)
    ) %>%
    group_by(data_name, actionset_name, model_type, method_name) %>%
    unite(cell_str, sep = "\\\\", all_of(str_replace(cell_metric_names, "_cnt", "_pct"))) %>%
    mutate(cell_str = sprintf("\\cell{r}{%s}\n", cell_str)) %>%
    ungroup() %>%
    select(!actionset_name) %>%
    arrange(data_name,
            match(model_type, all_model_types),
            match(method_name, all_method_names)
    )

# create headers manually to avoid unique names issues
headers_df = table_df %>%
    mutate(model_type = model_type, method_name = str_to_lower(method_name)) %>%
    group_by(model_type) %>%
    distinct(method_name)

# top level columns (model type)
top_headers = headers_df %>%
    group_by(model_type) %>%
    count() %>%
    arrange(match(model_type, all_model_types)) %>%
    mutate(model_type = recode(model_type, !!!MODEL_TYPE_TITLES)) %>%
    pull(name = model_type) %>%
    prepend(c(" " = 2))

# bottom level columns (methods)
sub_headers = headers_df %>%
    mutate(method_name = str_to_lower(method_name)) %>%
    pull(method_name) %>%
    prepend(c("Dataset", "Metrics")) %>%
    recode(!!!METHOD_TITLES)

kable_df = table_df %>%
  mutate(
    metrics = "\\metricsguide{}",
    data_name = recode(data_name, !!!DATASET_TITLES)
  ) %>%
  pivot_wider(
    names_from = c(model_type, method_name),
    values_from = cell_str,
    names_sort = FALSE,
    names_glue = "{model_type}+{method_name}",
  )

overview_table = kable_df %>%
    kable(
        booktabs = TRUE,
        escape = FALSE,
        col.names = sub_headers,
        format = "latex",
        table.envir = NULL,
        linesep = ""
    ) %>%
    #kable_styling(latex_options = c("repeat_header", "scale_down", latex_table_env = NULL)) %>%
    add_header_above(top_headers, bold = FALSE, escape = FALSE) %>%
    column_spec(column = 1, latex_column_spec = "r") %>%
    column_spec(column = 2, latex_column_spec = "r") %>%
    column_spec(column = 3, latex_column_spec = "r") %>%
    column_spec(column = 4, latex_column_spec = "r") %>%
    row_spec(2:nrow(kable_df)-1, hline_after = TRUE, extra_latex_after = "\n")

overview_table

Warning message:
“`prepend()` was deprecated in purrr 1.0.0.
ℹ Please use append(after = 0) instead.”



\begin{tabular}{rrrrlllll}
\toprule
\multicolumn{2}{c}{ } & \multicolumn{3}{c}{LR{}} & \multicolumn{2}{c}{XGB{}} & \multicolumn{2}{c}{RF{}} \\
\cmidrule(l{3pt}r{3pt}){3-5} \cmidrule(l{3pt}r{3pt}){6-7} \cmidrule(l{3pt}r{3pt}){8-9}
Dataset & Metrics & \reach{} & \ar{} & \dice{} & \reach{} & \dice{} & \reach{} & \dice{}\\
\midrule
\ficoinfo{} & \metricsguide{} & \cell{r}{25.8\%\\0.0\%\\74.2\%\\\textbf{0.0\%}\\25.8\%\\\textbf{0.0\%}} & \cell{r}{---\\0.0\%\\83.3\%\\\textcolor{\pitfall}{41.5\%}\\16.7\%\\0.0\%} & \cell{r}{---\\0.0\%\\52.0\%\\\textcolor{\pitfall}{32.0\%}\\48.0\%\\\textcolor{\pitfall}{22.9\%}} & \cell{r}{25.8\%\\0.0\%\\74.2\%\\\textbf{0.0\%}\\25.8\%\\\textbf{0.0\%}} & \cell{r}{---\\0.0\%\\52.3\%\\\textcolor{\pitfall}{38.8\%}\\47.7\%\\\textcolor{\pitfall}{22.6\%}} & \cell{r}{34.1\%\\0.0\%\\65.9\%\\\textbf{0.0\%}\\34.1\%\\\textbf{0.0\%}} & \cell{r}{---\\0.0\%\\45.6\%\\\textcolor{\pitfall}{26.8\%}\\54.4\%\\\textcolor{\pitfall}{20.6\%}}\\
\midrule


\germaninfo{} & \metricsguide{}

In [8]:
table_df = cells_df %>%
    group_by(data_name, actionset_name, model_type, method_name) %>%
    unite(cell_str, sep = "\\\\", all_of(str_replace(cell_metric_names, "_cnt", "_pct"))) %>%
    mutate(cell_str = sprintf("\\cell{r}{%s}\n", cell_str)) %>%
    ungroup() %>%
    arrange(data_name,
            match(actionset_name, all_actionset_names),
            match(model_type, all_model_types))

# create headers manually to avoid unique names issues
headers_df = table_df %>%
    mutate(actionset_name = str_to_lower(actionset_name), method_name = str_to_lower(method_name)) %>%
    arrange(match(model_type, all_model_types),
            match(method_name, all_method_names))

# create headers manually to avoid unique names issues
headers_df = table_df %>%
    mutate(actionset_name = actionset_name, method_name = str_to_lower(method_name)) %>%
    group_by(actionset_name) %>%
    distinct(method_name)

# top level columns (actionability)
top_headers = headers_df %>%
    group_by(actionset_name) %>%
    count() %>%
    arrange(match(actionset_name, all_actionset_names)) %>%
    mutate(actionset_name = recode(actionset_name, !!!ACTIONSET_TITLES)) %>%
    pull(name = actionset_name) %>%
    prepend(c(" " = 3))

# bottom level columns (methods)
sub_headers = headers_df %>%
    mutate(method_name = str_to_lower(method_name)) %>%
    pull(method_name) %>%
    prepend(c("Dataset", "Model Type", "Metrics")) %>%
    recode(!!!METHOD_TITLES)

kable_df = table_df %>%
    mutate(
           metrics = "\\metricsguide{}",
           model_type = recode(model_type, !!!MODEL_TYPE_TITLES),
           data_name = recode(data_name, !!!DATASET_TITLES)
           ) %>%
    pivot_wider(
        names_from = c(actionset_name, method_name),
        values_from = cell_str,
        names_sort = FALSE,
        names_glue = "{actionset_name}+{method_name}",
    )

overview_table = kable_df %>%
    kable(
        booktabs = TRUE,
        escape = FALSE,
        col.names = sub_headers,
        format = "latex",
        table.envir = NULL,
        linesep = ""
    ) %>%
    #kable_styling(latex_options = c("repeat_header", "scale_down", latex_table_env = NULL)) %>%
    add_header_above(top_headers, bold = FALSE, escape = FALSE) %>%
    column_spec(column = 1, latex_column_spec = "r") %>%
    column_spec(column = 2, latex_column_spec = "r") %>%
    column_spec(column = 3, latex_column_spec = "r") %>%
    row_spec(2:nrow(kable_df)-1, hline_after = TRUE, extra_latex_after = "\n")

overview_table


\begin{tabular}{rrrlllllllll}
\toprule
\multicolumn{3}{c}{ } & \multicolumn{3}{c}{Simple} & \multicolumn{3}{c}{Separable} & \multicolumn{3}{c}{Actual} \\
\cmidrule(l{3pt}r{3pt}){4-6} \cmidrule(l{3pt}r{3pt}){7-9} \cmidrule(l{3pt}r{3pt}){10-12}
Dataset & Model Type & Metrics & \ar{} & \dice{} & \reach{} & \ar{} & \dice{} & \reach{} & \ar{} & \dice{} & \reach{}\\
\midrule
\ficoinfo{} & \LR{} & \metricsguide{} & \cell{r}{---\\0.0\%\\99.9\%\\96.0\%\\0.1\%\\0.0\%} & \cell{r}{---\\0.0\%\\59.9\%\\45.9\%\\40.1\%\\16.7\%} & \cell{r}{25.8\%\\0.0\%\\74.2\%\\0.0\%\\25.8\%\\0.0\%} & \cell{r}{---\\0.0\%\\83.3\%\\41.5\%\\16.7\%\\0.0\%} & \cell{r}{---\\0.0\%\\51.6\%\\32.3\%\\48.4\%\\23.5\%} & \cell{r}{25.8\%\\0.0\%\\74.2\%\\0.0\%\\25.8\%\\0.0\%} & \cell{r}{---\\0.0\%\\83.3\%\\41.5\%\\16.7\%\\0.0\%} & \cell{r}{---\\0.0\%\\52.0\%\\32.0\%\\48.0\%\\22.9\%} & \cell{r}{25.8\%\\0.0\%\\74.2\%\\0.0\%\\25.8\%\\0.0\%}\\
\midrule


\ficoinfo{} & \XGB{} & \metricsguide{} & NA & \cell{r}{---\\0.0\%\\55.3\%\\43.0\%\

In [9]:
# print metric titles
cell_metric_titles = METRIC_TITLES[cell_metric_names]
metrics_cmd = paste0("\\renewcommand{\\metricsguide}[0]{\\cell{r}{",paste0(cell_metric_titles, collapse = "\\\\"), "}}")
metrics_cmd

[1] "\\renewcommand{\\metricsguide}[0]{\\cell{r}{Certifies no recourse\\\\Abstains/inconclusive\\\\Finds action\\\\Loophole\\\\Finds no action\\\\Blindspot}}"

In [10]:
model_data <- read_csv("../results/model_stats.csv", show_col_types = FALSE)

model_data %>%
    filter(action_set_name == "complex_nD") %>%
    select(c("data_name", "model_type", "train_auc", "test_auc", "train_error", "test_error")) %>%
    mutate(
        model_type = recode(model_type, !!!MODEL_TYPE_TITLES),
        data_name = sprintf("\\textfn{%s}", data_name)
    ) %>%

    kable(
        booktabs = TRUE,
        escape = FALSE,
        col.names = c("Dataset", "Model", "Train", "Test", "Train", "Test"),
        format = "latex",
        table.envir = NULL,
        linesep = ""
    ) %>%
    # kable_styling(latex_options = c("repeat_header", "scale_down", latex_table_env = NULL))
    add_header_above(c(" " = 2, "AUC" = 2, "Error" = 2), bold = FALSE, escape = FALSE)
    # column_spec(column = 1, latex_column_spec = "r") %>%
    # column_spec(column = 2, latex_column_spec = "r") %>%
    # row_spec(2:nrow(kable_df)-1, hline_after = TRUE, extra_latex_after = "\n")


\begin{tabular}{llrrrr}
\toprule
\multicolumn{2}{c}{ } & \multicolumn{2}{c}{AUC} & \multicolumn{2}{c}{Error} \\
\cmidrule(l{3pt}r{3pt}){3-4} \cmidrule(l{3pt}r{3pt}){5-6}
Dataset & Model & Train & Test & Train & Test\\
\midrule
\textfn{fico} & \LR{} & 0.7723 & 0.7882 & 0.2774 & 0.2774\\
\textfn{fico} & \XGB{} & 0.7721 & 0.7880 & 0.2783 & 0.2783\\
\textfn{fico} & \RF{} & 0.8593 & 0.7853 & 0.2877 & 0.2877\\
\textfn{german} & \LR{} & 0.8193 & 0.7602 & 0.2350 & 0.2350\\
\textfn{german} & \XGB{} & 0.8191 & 0.7614 & 0.2300 & 0.2300\\
\textfn{german} & \RF{} & 0.9708 & 0.7937 & 0.2350 & 0.2350\\
\textfn{givemecredit} & \LR{} & 0.8418 & 0.8437 & 0.0681 & 0.0681\\
\textfn{givemecredit} & \XGB{} & 0.8418 & 0.8438 & 0.0681 & 0.0681\\
\textfn{givemecredit} & \RF{} & 0.8626 & 0.8389 & 0.0681 & 0.0681\\
\bottomrule
\end{tabular}

In [12]:
data_stats <- read_csv("../results/data_stats.csv", show_col_types = FALSE) %>%
    filter(action_set_name == "complex_nD")

data_stats

data_name,action_set_name,n,d
<chr>,<chr>,<dbl>,<dbl>
fico,complex_nD,5842,43
german,complex_nD,1000,36
givemecredit,complex_nD,120268,23
